In [ ]:
# 데이터 구조
import pandas as pd
# 행렬 연산
import numpy as np
# 데이터 시각화
import matplotlib.pyplot as plt
import matplotlib
# 데이터 분할: train, test
from sklearn.model_selection import train_test_split
# 로지스틱 회귀모델
from statsmodels.api import Logit
# 평가 함수
from sklearn.metrics import accuracy_score
# confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
# 한글 폰트 설정
matplotlib.rc("font", family = "Malgun Gothic")
# 음수 표시 설정
matplotlib.rc("axes", unicode_minus = False)

In [ ]:
df_raw = pd.read_csv("../data/bank.csv")
df_raw.head()

In [ ]:
# 결측치 확인
df_raw.isnull().sum(axis = 0)

In [ ]:
# fillna: 결측치 대체 함수
# JOB 변수의 결측치는 "Other"로 대체
df_raw["JOB"].fillna("Other", inplace = True)
# 숫자형 변수의 결측치는 해당 변수의 평균값으로 대체
df_raw.fillna(df_raw.mean(), inplace=True)

In [ ]:
# 학습용/평가용 데이터 분리
df_train, df_test = train_test_split(df_raw, test_size = 0.3, random_state = 1234)
print("학습용 데이터 크기 : {}".format(df_train.shape))
print("평가용 데이터 크기 : {}".format(df_test.shape))

In [ ]:
# from_formula 함수를 이용하여 변수 역할 지정
log_model = Logit.from_formula("""BAD ~ LOAN + MORTDUE + VALUE + C(REASON) + C(JOB) + YOJ +
DEROG + DELINQ + CLAGE + NINQ + CLNO + DEBTINC""" , df_train)
# 모델 적용
log_result = log_model.fit()
# 결과 출력
print(log_result.summary())

In [ ]:
# 유의하지 않은 변수를 반복적으로 제거한 후 모델 수립
log_model = Logit.from_formula("""BAD ~ C(JOB) + DEROG +
DELINQ + CLAGE + NINQ + CLNO + DEBTINC""" , df_train)
# 모델 적용
log_result = log_model.fit()
# 결과 출력
print(log_result.summary())

In [ ]:
# 학습용 데이터 예측
y_pred = log_result.predict(df_train)
# 0과 1의 값을 가진 class로 변환
y_pred_class = (y_pred > 0.5).astype(int)
print("학습용 데이터 정확도: {0:.3f}".format(accuracy_score(df_train["BAD"], y_pred_class)))
# 평가용 데이터 예측
y_pred = log_result.predict(df_test)
y_pred_class = (y_pred > 0.5).astype(int)
print("평가용 데이터 정확도: {0:.3f}".format(accuracy_score(df_test["BAD"], y_pred_class)))

In [ ]:
# 실제 학습용 데이터의 목표변수와 예측값 비교
print("Confusion Matrix: \n{}".format(confusion_matrix(df_test["BAD"],y_pred_class)))

In [ ]:
# 실제 학습용 데이터의 목표변수와 예측값 비교
print("Classification Report: ", classification_report(df_test["BAD"], y_pred_class))

In [ ]:
# 설명변수 중요도
df_logistic_coef = pd.DataFrame({"Coef": log_result.params.values[1:]}, index = log_model.exog_names[1:])
df_logistic_coef.plot.barh(y = "Coef")